In [1]:
import os 
os.chdir('..')

import numpy as np
import pandas as pd
import seaborn as sns
import tellurium as te

import matplotlib.pyplot as plt
from src.odbm.odbm_main import ModelBuilder
from mpl_toolkits.axes_grid1 import make_axes_locatable
from Bio.KEGG.KGML import KGML_parser

In [105]:
os.getcwd()

'/workspaces/ECFERS'

Filter by Kcat/KM

In [3]:
rxn = pd.read_csv('src/frenda_brenda/Files/KEGG_Filtered/Reactions_M1.csv')
spc = pd.read_csv('src/frenda_brenda/Files/KEGG_Filtered/SpeciesBaseMechanism_biosyn.csv')

In [4]:
xml_file = 'ko01100.xml'
pathway = KGML_parser.read(open(xml_file, 'r'))

In [ ]:
# issue? some reactions have more than one "primary" substrate or product. we'll have to account for this
# pull the primary substrate from XMl. decide if it is on the product or substrate side of our CSV file
# depending on that, pull the KCatF or the KCatR
# pull the respective KM

# will have to group by EC ID at some point. some reaction IDs from KEGG are not in the visualization XML file

In [21]:
reaction_dict = {}

for r in list(pathway.reactions):
    reaction_dict[r.id] = r.name.replace("rn:", "").split(' ')

In [25]:
def find_keys_by_value_substring(reaction_dict, substring):
    matching_keys = [key for key, value in reaction_dict.items() if substring in value]
    return matching_keys

In [53]:
sub_dict = {}
prod_dict = {}

for r in list(pathway.reactions):
    sub_dict[r.id] = [s for s in r._substrates]
    prod_dict[r.id] = [p for p in r._products]

In [63]:
rid_to_subnames = {}
rid_to_prodnames = {}
for RID in rxn['Reaction ID']:
    matching_keys = find_keys_by_value_substring(reaction_dict, RID)
    if matching_keys:
        subnames_list = []
        prodnames_list = []
        for key in matching_keys:
            subid = sub_dict[key]
            prodid = prod_dict[key]
            for sub in subid:
                subname = pathway.entries[sub].name.replace('cpd:', '')
                subnames_list.append(subname)
            for prod in prodid:
                prodname = pathway.entries[prod].name.replace('cpd:', '')
                prodnames_list.append(prodname)
        rid_to_subnames[RID] = subnames_list
        rid_to_prodnames[RID] = prodnames_list
    #else:
        # what do we do if they are not in the XML file?
print(rid_to_subnames)
print(rid_to_prodnames)

{'R00209': ['C00022'], 'R00342': ['C00149', 'C00149'], 'R00199': ['C00074'], 'R00341': ['C00036', 'C00074'], 'R00945': ['C00101', 'C00037', 'C00143'], 'R09099': ['C00065'], 'R01082': ['C00122'], 'R00355': ['C00036', 'C00036', 'C00025'], 'R00345': ['C00036', 'C00074', 'C00011'], 'R00200': ['C00074'], 'R01138': ['C00131'], 'R01858': ['C00361'], 'R02320': ['C00074'], 'R00673': ['C00463'], 'R00216': ['C00149'], 'R00217': ['C00036'], 'R05605': ['C00118'], 'R00471': ['C00048'], 'R00226': ['C06010'], 'R08648': ['C00109', 'C00022'], 'R00220': ['C00022'], 'R00590': ['C00065'], 'R00214': ['C00149'], 'R01811': ['C00270'], 'R01731': ['C00254'], 'R00704': ['C00022'], 'R00351': ['C00036', 'C00024'], 'R00344': ['C00036'], 'R00519': ['C00058']}
{'R00209': ['C00024'], 'R00342': ['C00036', 'C00036'], 'R00199': ['C00022'], 'R00341': ['C00074', 'C00036'], 'R00945': ['C00143', 'C00101', 'C00065'], 'R09099': ['C04377'], 'R01082': ['C00149'], 'R00355': ['C00049', 'C00049'], 'R00345': ['C00074', 'C00036'], 'R

In [67]:
rid_to_subnames['R00209']

['C00022']

In [78]:
rid_to_subnames['R00209'][0] in rxn[rxn['Label']=='R23']['Substrates'].values[0]

True

In [ ]:
row

In [92]:
rid_to_subnames[row['Reaction ID']][0]

'C00022'

'C00003; C00010; C00022'

In [104]:
rid_to_subnames[row['Reaction ID']][0]

'C00149'

In [103]:
for index, row in rxn.iterrows():
    try:
        primarysub = rid_to_subnames[row['Reaction ID']]
        primaryprod = rid_to_prodnames[row['Reaction ID']]
    except KeyError:
        continue

    if rid_to_subnames[row['Reaction ID']][0] in row['Substrates']: # this is not dealing with multiple substrates yet
        print(f'this is a substrate')
        # kcat = forward kcat
    else:
        print(f'this is NOT a substrate')
        # check if it is a product. if yes, kcat = reverse kcat. if no, then that is a problem?

    # grab the km associated with this primary substrate
    # calculate kcat/km
    # redo this same logic with the primary product
    # do we sum these kcat/km metrics? do we prioritize only having one be high?
    
    if rid_to_subnames[row['Reaction ID']][0] in row['Products']: # this is not dealing with multiple substrates yet
        print(f'this is a product')
    else:
        print(f'this is NOT a product')

this is a substrate
this is NOT a product
this is a substrate
this is NOT a product
this is a substrate
this is NOT a product
this is a substrate
this is NOT a product
this is NOT a substrate
this is a product
this is a substrate
this is NOT a product
this is NOT a substrate
this is a product
this is a substrate
this is NOT a product
this is NOT a substrate
this is a product
this is NOT a substrate
this is a product
this is a substrate
this is NOT a product
this is NOT a substrate
this is a product
this is a substrate
this is NOT a product
this is NOT a substrate
this is a product
this is NOT a substrate
this is a product
this is NOT a substrate
this is a product
this is a substrate
this is NOT a product
this is a substrate
this is NOT a product
this is NOT a substrate
this is a product
this is NOT a substrate
this is a product
this is a substrate
this is NOT a product
this is a substrate
this is NOT a product
this is NOT a substrate
this is a product
this is a substrate
this is NOT a 

Filter by KEGG Number

In [76]:
rnum = pd.read_csv('src/frenda_brenda/Files/Reaction_numfilt.csv')
sbmnum = pd.read_csv('src/frenda_brenda/Files/SpeciesBaseMechanism_numfilt.csv')

In [77]:
# Extract the number after 'R' in the 'Reaction ID' column and convert it to integer
rnum['Reaction Number'] = rnum['Reaction ID'].str.extract(r'R(\d+)').astype(int)

# For rows with the same value in 'EC', keep the one with the lowest 'Reaction Number'
df_result = rnum.loc[rnum.groupby('EC')['Reaction Number'].idxmin()]

# Drop the 'Reaction Number' column as it's not needed in the final output
df_result = df_result.drop(columns=['Reaction Number'])
df_result['Label Number'] = df_result['Label'].str.extract(r'R(\d+)').astype(int)

# Sort the DataFrame by the 'Label Number'
rnum_sorted = df_result.sort_values(by='Label Number')

# Drop the 'Label Number' column after sorting
rnum_sorted = rnum_sorted.drop(columns=['Label Number'])

In [78]:
# Condition 1: If 'Label' starts with 'C', check if it exists in rnum['Label']
mask_label = sbmnum['Label'].str.startswith('C') & (
    sbmnum['Label'].isin(rnum_sorted['Substrates']) | sbmnum['Label'].isin(rnum_sorted['Products'])
)

# Condition 2: If 'EC' is not NaN, check if it exists in rnum['EC']
mask_ec = sbmnum['EC'].notna() & sbmnum['EC'].isin(rnum_sorted['EC'])

# Combine both conditions, keep rows where either condition is true
mask_combined = mask_label | mask_ec

# Apply the mask to filter sbmnum
sbmnum_filtered = sbmnum[mask_combined]

In [79]:
# Split the values in 'Substrates' and 'Products' columns by semicolons and expand them into lists
substrates_expanded = rnum_sorted['Substrates'].str.split(';').explode().str.strip()
products_expanded = rnum_sorted['Products'].str.split(';').explode().str.strip()

# Combine the expanded 'Substrates' and 'Products' into a single Series
all_expanded_ids = pd.concat([substrates_expanded, products_expanded])

# Condition 1: If 'Label' starts with 'C', check if it exists in the expanded IDs
mask_label = sbmnum['Label'].str.startswith('C') & sbmnum['Label'].isin(all_expanded_ids)

# Condition 2: If 'EC' is not NaN, check if it exists in rnum['EC']
mask_ec = sbmnum['EC'].notna() & sbmnum['EC'].isin(rnum_sorted['EC'])

# Combine both conditions, keep rows where either condition is true
mask_combined = mask_label | mask_ec

# Apply the mask to filter sbmnum
sbmnum_filtered = sbmnum[mask_combined]

# Apply the mask to filter sbmk (you can combine it with other conditions if needed)
sbmnum_filtered = sbmnum[mask_combined]

In [80]:
rnum_sorted.to_csv('src/frenda_brenda/Files/Reaction_numfilt.csv')
sbmnum_filtered.to_csv('src/frenda_brenda/Files/SpeciesBaseMechanism_numfilt.csv')

Filtering by Kcat

In [81]:
rk = pd.read_csv('src/frenda_brenda/Files/Reaction_kfilt.csv')
sbmk = pd.read_csv('src/frenda_brenda/Files/SpeciesBaseMechanism_kfilt.csv')

In [82]:
rk['Kcat_F'] = rk['Kcat'].str.extract(r'Kcat_F:\s*([\d\.]+)').astype(float)
rk['Kcat_R'] = rk['Kcat'].str.extract(r'Kcat_R:\s*([\d\.]+)').astype(float)

# Get the maximum value between Kcat_F and Kcat_R for each row
rk['Max_Kcat'] = rk[['Kcat_F', 'Kcat_R']].max(axis=1)

# For rows with the same value in 'EC', keep the one with the largest 'Max_Kcat'
df_result = rk.loc[rk.groupby('EC')['Max_Kcat'].idxmax()]

# Drop the extra columns
df_result = df_result.drop(columns=['Kcat_F', 'Kcat_R', 'Max_Kcat'])

df_result['Label Number'] = df_result['Label'].str.extract(r'R(\d+)').astype(int)

# Sort the DataFrame by the 'Label Number'
rk_sorted = df_result.sort_values(by='Label Number')

# Drop the 'Label Number' column after sorting
rk_sorted = rk_sorted.drop(columns=['Label Number'])

In [83]:
# Split the values in 'Substrates' and 'Products' columns by semicolons and expand them into lists
substrates_expanded = rk_sorted['Substrates'].str.split(';').explode().str.strip()
products_expanded = rk_sorted['Products'].str.split(';').explode().str.strip()

# Combine the expanded 'Substrates' and 'Products' into a single Series
all_expanded_ids = pd.concat([substrates_expanded, products_expanded])

# Condition 1: If 'Label' starts with 'C', check if it exists in the expanded IDs
mask_label = sbmk['Label'].str.startswith('C') & sbmk['Label'].isin(all_expanded_ids)

# Condition 2: If 'EC' is not NaN, check if it exists in rnum['EC']
mask_ec = sbmk['EC'].notna() & sbmk['EC'].isin(rk_sorted['EC'])

# Combine both conditions, keep rows where either condition is true
mask_combined = mask_label | mask_ec

# Apply the mask to filter sbmnum
sbmk_filtered = sbmk[mask_combined]

# Apply the mask to filter sbmk (you can combine it with other conditions if needed)
sbmk_filtered = sbmk[mask_combined]

In [84]:
rk_sorted.to_csv('src/frenda_brenda/Files/Reaction_kfilt.csv')
sbmk_filtered.to_csv('src/frenda_brenda/Files/SpeciesBaseMechanism_kfilt.csv')

Filter by inclusion in KEGG map

In [122]:
from Bio.KEGG.KGML import KGML_parser

In [106]:
rxn = pd.read_csv('src/frenda_brenda/Files/Model Reduction Approaches/Reaction_mostcomplete.csv')
sbm = pd.read_csv('src/frenda_brenda/Files/Model Reduction Approaches/SpeciesBaseMechanisms_mostcomplete.csv')

In [159]:
filtered_rxn = pd.read_csv('src/frenda_brenda/Files/Model Reduction Approaches/Reaction_filtByMap.csv')

In [109]:
xml_file = 'ko01100.xml'
pathway = KGML_parser.read(open(xml_file, 'r'))

In [118]:
mapped_rxns = []
for reaction in pathway.reactions:
    for r in reaction.name.replace("rn:","").split(' '):
        mapped_rxns.append(r)

In [126]:
# Create a new DataFrame that only includes rows where 'Reaction ID' is in mapped_rxns
filtered_rxn = rxn[rxn['Reaction ID'].isin(mapped_rxns)].copy()

In [160]:
# Split the values in 'Substrates' and 'Products' columns by semicolons and expand them into lists
substrates_expanded = filtered_rxn['Substrates'].str.split(';').explode().str.strip()
products_expanded = filtered_rxn['Products'].str.split(';').explode().str.strip()
inhibitors_expanded = filtered_rxn['Inhibitors'].str.split(';').explode().str.strip().dropna()

In [161]:
# Combine the expanded 'Substrates' and 'Products' into a single Series
all_expanded_ids = pd.concat([substrates_expanded, products_expanded, inhibitors_expanded])

In [162]:
# Condition 1: If 'Label' starts with 'C', check if it exists in the expanded IDs
mask_label = sbm['Label'].str.startswith('C') & sbm['Label'].isin(all_expanded_ids)

# Condition 2: If 'EC' is not NaN, check if it exists in rnum['EC']
mask_ec = sbm['EC'].notna() & sbm['EC'].isin(filtered_rxn['EC'])

In [163]:
# Combine both conditions, keep rows where either condition is true
mask_combined = mask_label | mask_ec

In [164]:
# Apply the mask to filter sbmnum
sbm_filtered = sbm[mask_combined]

In [152]:
# filtered_rxn.to_csv('src/frenda_brenda/Files/Model Reduction Approaches/Reaction_filtByMap.csv')

In [165]:
sbm_filtered.to_csv('src/frenda_brenda/Files/Model Reduction Approaches/SpeciesBaseMechanism_filtByMap.csv')

Examine the excluded reactions

In [156]:
diff = rxn.merge(filtered_rxn, how='left', indicator=True)

# Filtering rows that are only in rxn (i.e., not in filtered_rxn)
diff_in_rxn_only = diff[diff['_merge'] == 'left_only']

# Drop the merge indicator column if it's no longer needed
diff_in_rxn_only = diff_in_rxn_only.drop(columns=['_merge'])

In [158]:
diff_in_rxn_only.to_csv('src/frenda_brenda/Files/Model Reduction Approaches/Exclused_Reactions.csv')

Remove reactions and species for enzymes which have zero concentration

In [166]:
rxn = pd.read_csv('src/frenda_brenda/Files/Model Reduction Approaches/Reaction_filtByMap_withStoich.csv')
sbm = pd.read_csv('src/frenda_brenda/Files/Model Reduction Approaches/SpeciesBaseMechanism_filtByMap.csv')

In [175]:
sbm_filtered = sbm[sbm['StartingConc'] != 0]

In [176]:
ec_to_remove = sbm[sbm['StartingConc'] == 0]['EC'].unique()

rxn_filtered = rxn[~rxn['EC'].isin(ec_to_remove)]

In [179]:
sbm_filtered.to_csv('src/frenda_brenda/Files/Model Reduction Approaches/241021_SpeciesBaseMechanism.csv')

In [180]:
rxn_filtered.to_csv('src/frenda_brenda/Files/Model Reduction Approaches/241021_Reaction.csv')